In [1]:
from keras.datasets import mnist
(train_figures_X, train_y), (test_figures_X, test_y) = mnist.load_data(path='./datasets')
print('train_X shape:', train_figures_X.shape)
print('train_y shape:', train_y.shape)
print('test_X shape:', test_figures_X.shape)
print('test_y shape:', test_y.shape)
print('train_X[0]:', train_figures_X[0])
print('train_y[0]:', train_y[0])
print('test_X[0]:', test_figures_X[0])
print('test_y[0]:', test_y[0])
print('train_X[0].shape:', train_figures_X[0].shape)
print('train_y[0].shape:', train_y[0].shape)
print('test_X[0].shape:', test_figures_X[0].shape)
print('test_y[0].shape:', test_y[0].shape)
print(type(train_figures_X[0]))
print(type(train_y[0].shape))
print(type(train_figures_X))
print(type(train_y))
image=train_figures_X[0]



train_X shape: (60000, 28, 28)
train_y shape: (60000,)
test_X shape: (10000, 28, 28)
test_y shape: (10000,)
train_X[0]: [[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   3  18  18  18 126 136
  175  26 166 255 247 127   0   0   0   0]
 [  0   0   0   0   0   0   0   0  30  36  94 154 170 253 253 253 253 253
  225 172 253 242 195  64   0   0   0   0]
 [  0   0   0   0   0   0   0  49 238 253 253 253 253 253 25

In [2]:
import numpy as np
import cv2
import pandas as pd


In [3]:
def HarrisCorner(image_array:np.ndarray)->int:
    '''Harris角点特征提取'''
    corners_mark = cv2.cornerHarris(image_array, blockSize=5,ksize=3,k=0.1)
    corners_mark=cv2.erode(corners_mark,None)#腐蚀
    corners_mark=cv2.dilate(corners_mark,None)#膨胀
    # print(np.size(corner_image))
    # print(np.size(image))


    corners_image=image_array.copy()#注意这里一定要深拷贝
    corners_image[corners_mark>0.005*corners_mark.max()]=255
    '''角点坐标数组'''
    # cv2.imshow('corners_image',corners_image)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()

    count_array=np.zeros((image_array.shape[0],image_array.shape[1]))
    '''计数使用的数组'''
    count_array[corners_mark>0.005*corners_mark.max()]=1
    # print(count_array[:,0])
    corner_num:int=int(np.sum(count_array))
    # print(corner_num)
    return corner_num

def getContours(image_array:np.ndarray)->tuple:
    '''获取轮廓'''
    # thresh, image_thresh = cv2.threshold(image_array, 127, 255, 0)
    image_thresh = cv2.adaptiveThreshold(image_array,255,cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,13,-10)
    image_thresh=cv2.erode(image_thresh,None)#腐蚀
    image_thresh=cv2.dilate(image_thresh,None)#膨胀
    contours, hierarchy=cv2.findContours(image_thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)
    contours:tuple
    '''轮廓'''
    # hierarchy输出各个轮廓的继承关系
    # contours_image=cv2.drawContours(image_thresh,contours,-1,(127,255,127),2)
    # cv2.imshow('contours_image',contours_image)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    return contours

def getContoursNumPerimeterAndArea(image_array:np.ndarray)->tuple:
    '''获取轮廓条数、长度和面积，其中返回的面积是轮廓圈出来的面积占总面积的比例'''
    contours:tuple=getContours(image_array)
    # print(len(contours))#轮廓数量
    # print(contours)
    #每一条轮廓是一个array。对于每一条轮廓由若干个点组成
    contours_num:int=len(contours)

    #计算所有轮廓周长
    # contour_len=cv2.arcLength(contours[0],closed=True)
    # print(contour_len)#第一条轮廓长
    contours_len:float=0.0
    '''轮廓总长度'''
    for contour in contours:
        contour_len=cv2.arcLength(contours[0],closed=True)
        '''每个轮廓的长度'''
        contours_len=contours_len+contour_len
    # print(contours_len)#轮廓总长

    #计算所有轮廓围成的面积
    contours_area:float=0.0
    for contour in contours:
        contour_area=cv2.contourArea(contour,oriented=False)
        contours_area=contours_area+contour_area
    # print(contours_area)#轮廓围成的面积
    contours_area_normalized:float=contours_area/(image_array.shape[0]*image_array.shape[1])
    # print(contours_area_normalized)#归一化轮廓围成的面积
    return (contours_num,contours_len,contours_area_normalized)

def CannyEdgesPointsNum(image_array:np.ndarray)->int:
    '''canny边缘点的个数'''
    canny_low_threshold=30
    canny_ratio=3
    canny_kernel_size=3
    canny_edges:np.ndarray=cv2.Canny(image_array,canny_low_threshold,canny_low_threshold*canny_ratio,apertureSize=canny_kernel_size)

    # cv2.imshow('canny',canny_edges)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    # np.set_printoptions(threshold=np.inf)
    # print(canny_edges[0:10,:])
    # print(type(canny_edges))
    # print(canny_edges.shape)
    #获得的数据不是有层次的，所以只能统计点的个数
    canny_edges_points_num=int(np.sum(canny_edges)/255)
    # print(canny_edges_points_num)
    return canny_edges_points_num


In [4]:
image_array=train_figures_X[0]
print(HarrisCorner(image_array))
print(getContoursNumPerimeterAndArea(image_array))
print(CannyEdgesPointsNum(image_array))

163
(3, 102.42640614509583, 0.08418367346938775)
97


In [5]:
def getFeatures(X:np.ndarray,y:np.ndarray)->pd.DataFrame:
    columns_index:list=['HarrisCorner','ContoursNum','ContoursPerimeter','ContoursArea','CannyEdges','NumClassification']
    features=pd.DataFrame(columns=columns_index,dtype='object')
    for index in range(X.shape[0]):
        image_array=X[index]
        num_classification:int=y[index]
        corner_num=HarrisCorner(image_array)
        contours_num,contours_perimeter,contours_area=getContoursNumPerimeterAndArea(image_array)
        contours_num:int
        contours_perimeter:float
        contours_area:float
        canny_edges_points_num:int=CannyEdgesPointsNum(image_array)
        features_line=pd.Series([corner_num,contours_num,contours_perimeter,contours_area,canny_edges_points_num,num_classification],index=columns_index,dtype='object')
        # print(features_line)
        # print(features_line)
        features=pd.concat([features,features_line.to_frame().T],ignore_index=True,axis=0)
    return features
def getFeaturesX(features:pd.DataFrame)->np.ndarray:
    features_X:np.ndarray=features.loc[:,'HarrisCorner':'CannyEdges'].values.astype('float')
    features_X=np.insert(arr=features_X,obj=0,values=1,axis=1)
    '''在最前面插入一列1，用于计算偏置'''
    return features_X


In [6]:
train_features:pd.DataFrame=getFeatures(train_figures_X,train_y)
test_features:pd.DataFrame=getFeatures(test_figures_X,test_y)
train_features_X:np.ndarray=getFeaturesX(train_features)
'''I行D+1列'''
test_features_X:np.ndarray=getFeaturesX(test_features)
'''I行D+1列'''

'I行D+1列'

In [22]:
print(train_features)

      HarrisCorner ContoursNum ContoursPerimeter ContoursArea CannyEdges  \
0              163           3        102.426406     0.084184         97   
1               83           3         91.455844     0.076531        107   
2              105           3              36.0      0.02551         84   
3               56           1         52.627417     0.066327         60   
4              113           2              24.0     0.023597         81   
...            ...         ...               ...          ...        ...   
59995          162           2              20.0     0.015306        109   
59996          181           3         80.485281     0.079719         91   
59997          113           2              20.0     0.026148         84   
59998          110           1              10.0     0.007653         94   
59999          146           2              16.0     0.010204         93   

      NumClassification  
0                     5  
1                     0  
2        

In [7]:
N:int=10
'''类别数为10'''
D:int=5
'''维数为5，记得需要多加一维'''

def softmax(x:np.ndarray)->np.ndarray:
    e_x = np.exp(x-np.max(x))
    return e_x / e_x.sum(axis=0)

def logSoftmax(x:np.ndarray)->np.ndarray:
    tmp=x-np.max(x)
    return tmp-np.log(np.sum(np.exp(tmp)))

def multiClassLogisticRegression(phy:np.ndarray,features_X:np.ndarray)->np.ndarray:
    '''多分类逻辑回归'''
    #前者是D+1行N列，后者是D+1行I列，两者相乘得到N行I列
    return softmax(np.dot(phy.T,features_X.T))



In [20]:
import winsound
duration = 1000 # 持续时间/ms
frequency = 500 # 频率/Hz



def multiClassLogisticRegressionTrain(features_X:np.ndarray,y:np.ndarray,phy:np.ndarray,learning_rate:float,iteration_num:int)->np.ndarray:
    '''多分类逻辑回归训练'''
    I:int=features_X.shape[0]
    '''训练样本数'''
    g_n:np.ndarray=np.zeros((D+1,N))
    '''梯度'''
    H:np.ndarray=np.zeros(shape=(N,N,D+1,D+1))
    '''Hessian矩阵'''
    for iteration_index in range(iteration_num):
        #实现算法9.8
        L=0
        '''目标函数'''

        #for 循环内部是copilot帮忙写的，我检查了一遍，没有什么问题，非常巧妙
        for i in range(I):
            # print(f"---------------------------第{i}个样本—------------------------")
            # print(f"前梯度：{g_n}")
            # # print(f"前Hessian矩阵：{H}")
            # print(f"前phy：{phy}")
            x_i:np.ndarray=features_X[i]
            '''第i个样本'''
            y_i=softmax(np.dot(phy.T,x_i))
            w_i:int=y[i]
            '''第i个样本的类别'''
            w_i_vector=np.zeros(N)
            '''第i个样本的类别向量，N行1列'''
            w_i_vector[w_i]=1
            log_y_i=logSoftmax(np.dot(phy.T,x_i))
            delta_L=-np.dot(w_i_vector,log_y_i)
            L=L+delta_L
            # print(f"softmax前：{np.dot(phy.T,x_i)}")
            # print(f"第{i}个样本，预测类别向量：{np.argmax(y_i)}，真实类别：{w_i}，损失：{delta_L}")
            # print(f"logSoftmax(np.dot(phy.T,x_i))：{log_y_i}")
            '''目标函数'''
            #后者是1行N列，前者是D+1行1列，两者相乘得到D+1行N列
            g_n=g_n+np.dot(x_i.reshape(D+1,1),((y_i-w_i_vector).reshape(1,N)))
            '''梯度'''
            H_n=np.zeros(shape=(N,N,D+1,D+1))
            kernel=np.dot(x_i.reshape(D+1,1),x_i.reshape(1,D+1))
            for n in range(N):
                for m in range(N):
                    H_n[m,n]=y_i[m]*(np.eye(N)[m,n]-y_i[n])*kernel
                    
            H=H+H_n
            for n in range(N):
                for m in range(N):
                    # print(H[m,n].shape)
                    # print(H[m,n])
                    # print(learning_rate*np.dot(np.linalg.inv(H[m,n]),g_n))
                    #前者是D+1行D+1列，后者是D+1行N列，两者相乘得到D+1行N列
                    try:
                        tmp=np.dot(np.linalg.inv(H[m,n]),g_n)
                        phy=phy-learning_rate*tmp#TODO 这个地方有问题
                    except Exception as e:
                        print(f'Exception:{e}')
                        print(f'H_n[m,n]:{H_n[m,n]}')
                        print(f'H[m,n]:{H[m,n]}')
                        print(f'y_i:{y_i}')
                        print(f'm:{m}')
                        print(f'n:{n}')
                        print(f'y_i[m]:{y_i[m]}')
                        print(f'y_i[n]:{y_i[n]}')
                        winsound.Beep(frequency, duration)
                        return
            # print(f"x_i={x_i}")
            # print(f"y_i={y_i}")
            # print(f"w_i={w_i}")
            # print(f"后梯度：{g_n}")
            # # print(f"后Hessian矩阵：{H}")
            # print(f"后phy：{phy}")
            # print(f"phy的形状：{phy.shape}")
            # print(f"H[m,n].shape:{H[m,n].shape}")
            # print(f"g_n.shape:{g_n.shape}")
        print(f'L={L}，第{iteration_index}次迭代')
    return phy

phy=np.ones((D+1,N))
phy=np.divide(phy,2)
'''特征矩阵'''
phy:np.ndarray=multiClassLogisticRegressionTrain(train_features_X,train_y,phy,1e-10,100)
winsound.Beep(frequency, duration)


L=5.0927807565081824e+22，第0次迭代
L=3.637265153289341e+23，第1次迭代
L=9.918351323677708e+23，第2次迭代


KeyboardInterrupt: 

In [ ]:
test_y_predict:np.ndarray=multiClassLogisticRegression(phy,test_features_X)

#统计
test_y_hut=np.argmax(test_y_predict,axis=0)
'''预测类别'''
test_y_telda=test_y-test_y_hut
'''差值'''
test_y_difference=np.zeros(test_y_telda.shape[0])
test_y_difference[np.nonzero(test_y_telda)]=1
'''差值不为0的地方置1'''
sum_difference=np.sum(test_y_difference)
'''差值不为0的个数'''
average_difference=sum_difference/test_y_telda.shape[0]
'''平均差值'''
print(f"平均差值：{average_difference}")

平均差值：0.902


C:\Users\LAPTOP_Lin-uCong\AppData\Local\Temp\ipykernel_48060\707241233.py:8: RuntimeWarning: invalid value encountered in divide
  return e_x / e_x.sum(axis=0)
